# Modelo: XGBoost com Tf-Idf

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

In [2]:
def calc_metricas(obs, pred):
    r2 = metrics.r2_score(obs, pred).round(3)
    mape = (100 * metrics.mean_absolute_percentage_error(obs, pred)).round(3)
    mae = metrics.mean_absolute_error(obs, pred).round(2)
    rmse = (metrics.mean_squared_error(obs, pred)**0.5).round(2)
    msle = metrics.mean_squared_log_error(obs, pred).round(3)
    
    return [r2, mape, mae, rmse, msle]

def print_avaliacao(obs, pred):
    print('R² = %.3f' % metrics.r2_score(obs, pred))
    print('MAPE = %.3f %%' % (100 * metrics.mean_absolute_percentage_error(obs, pred)))
    print('MAE = U$S %.2f' % (metrics.mean_absolute_error(obs, pred)))
    print('RMSE = U$S %.2f' % metrics.mean_squared_error(obs, pred)**0.5)
    print('MSLE = %.3f' % metrics.mean_squared_log_error(obs, pred))


In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


def prep_text(text):
    try:
        tokens = word_tokenize(text)
        words = [word.lower() for word in tokens if word.isalpha()]
        words = [word for word in words if not word in stop_words]
    except:
        return None
    return ' '.join(words)

In [4]:
dados_treino = pd.read_csv('../data/processed/train_data.csv')
dados_teste = pd.read_csv('../data/processed/test_data.csv')

## Preparação das colunas de texto

In [5]:
dados_treino.columns

Index(['name', 'category_1', 'category_2', 'category_3', 'item_condition_id',
       'brand_name', 'price', 'shipping', 'item_description', 'date', 'stock'],
      dtype='object')

In [6]:
dados_treino = dados_treino[['category_1', 'name', 'item_description', 'price']]
dados_treino.loc[:, 'name'] = dados_treino.apply(lambda x: prep_text(x['name']), axis=1)
dados_treino.loc[:, 'item_description'] = dados_treino.apply(lambda x: prep_text(x['item_description']), axis=1)
dados_treino['comb_name_description'] = dados_treino['name'] + ' ' + dados_treino['item_description']
# dados_treino.dropna(inplace=True)

dados_teste = dados_teste[['category_1', 'name', 'item_description', 'price']]
dados_teste.loc[:, 'name'] = dados_teste.apply(lambda x: prep_text(x['name']), axis=1)
dados_teste.loc[:, 'item_description'] = dados_teste.apply(lambda x: prep_text(x['item_description']), axis=1)
dados_teste['comb_name_description'] = dados_teste['name'] + ' ' + dados_teste['item_description']

## Apenas names como característica

In [ ]:
dados_treino1 = dados_treino[['category_1', 'name', 'price']]
dados_treino1.dropna(inplace=True)

dados_teste1 = dados_teste[['category_1', 'name', 'price']]
dados_teste1.dropna(inplace=True)

In [8]:
pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
X_treino = dados_treino1['name']
y_treino = np.log(dados_treino1['price'])
pipe.fit(X_treino, y_treino)

Pipeline(steps=[('tiidf',
                 TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 2))),
                ('xgboost',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                              predictor='auto', random_state=101, reg_alpha=0,
                              reg_lambda=1, ...))])

Tempo de treino: 2m 11.3s

In [ ]:
dados_teste1['pred'] = np.exp(pipe.predict(dados_teste1['name']))
dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo previsão: 11.8s

Avaliação geral

In [10]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.165
MAPE = 54.739 %
MAE = U$S 13.45
RMSE = U$S 35.05
MSLE = 0.372


Avaliação por categoria

In [11]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.172,0.134,0.153,0.156,0.040,0.135,0.026,0.070,0.053,0.180,0.127
MAPE,48.092,46.000,74.852,64.582,59.398,59.127,70.826,54.538,53.550,76.903,59.133
MAE,13.550,17.870,16.100,22.230,9.660,10.040,11.840,11.780,13.350,9.960,13.100
RMSE,35.610,37.290,46.640,60.480,23.740,20.680,27.070,22.470,28.890,22.720,28.300
MSLE,0.327,0.413,0.531,0.572,0.344,0.340,0.456,0.358,0.401,0.413,0.409


## Modelo por categoria usando apenas a coluna name

In [12]:
modelos = {}

for category in dados_treino.category_1.unique():
    pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
    modelos[category] = pipe
    X_treino = dados_treino1.query(f'category_1 == "{category}"')['name']
    y_treino = np.log(dados_treino1.query(f'category_1 == "{category}"')['price'])
    # pipe.fit(X_treino, y_treino)
    modelos[category].fit(X_treino, y_treino)


Tempo de treino: 2m 16.2s

In [13]:
dados_teste1 = dados_teste1.assign(pred=0)

In [14]:
for category in dados_teste1.category_1.unique():
    index = dados_teste1.query(f'category_1 == "{category}"').index
    dados_teste1.loc[index, 'pred'] = np.exp(modelos[category].predict(dados_teste1.loc[index]['name']))

dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo de previsão: 5.7s

Avaliação geral:

In [15]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.269
MAPE = 49.182 %
MAE = U$S 12.34
RMSE = U$S 32.81
MSLE = 0.310


Avaliação por categoria principal

In [16]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.235,0.376,0.232,0.316,0.100,0.214,0.140,0.261,0.291,0.243,0.095
MAPE,47.635,48.841,60.915,59.441,46.821,47.584,52.567,47.042,47.342,48.913,61.726
MAE,12.940,15.430,14.990,19.560,8.610,9.120,10.290,10.320,11.300,8.520,13.560
RMSE,34.240,31.640,44.420,54.440,22.990,19.720,25.440,20.020,25.010,21.820,28.810
MSLE,0.297,0.298,0.443,0.429,0.275,0.286,0.350,0.282,0.303,0.300,0.440


## Modelo geral usando coluna item_description

In [ ]:
dados_treino1 = dados_treino[['category_1', 'item_description', 'price']]
dados_treino1.dropna(inplace=True)

dados_teste1 = dados_teste[['category_1', 'item_description', 'price']]
dados_teste1.dropna(inplace=True)

In [18]:
pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                 ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
X_treino = dados_treino1['item_description']
y_treino = np.log(dados_treino1['price'])
pipe.fit(X_treino, y_treino)

Pipeline(steps=[('tiidf',
                 TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 2))),
                ('xgboost',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                              predictor='auto', random_state=101, reg_alpha=0,
                              reg_lambda=1, ...))])

Tempo de treino: 9m 37.8s

In [ ]:
dados_teste1['pred'] = np.exp(pipe.predict(dados_teste1['item_description']))
dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo de previsão: 20.8s

Avaliação geral

In [20]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.170
MAPE = 57.278 %
MAE = U$S 13.86
RMSE = U$S 34.96
MSLE = 0.398


Avaliação por categoria principal

In [21]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.173,0.096,0.146,0.189,0.049,0.110,0.047,0.028,0.078,0.183,0.065
MAPE,51.937,49.030,72.864,68.093,60.032,61.623,69.160,54.419,55.638,78.521,60.266
MAE,14.350,18.540,15.990,21.800,9.640,10.350,11.680,12.100,13.390,10.070,13.510
RMSE,35.600,38.100,46.870,59.280,23.630,20.980,26.810,22.980,28.530,22.640,29.280
MSLE,0.375,0.463,0.524,0.552,0.343,0.360,0.446,0.379,0.403,0.432,0.440


## Modelo por categoria usando item_description

In [22]:
modelos = {}

for category in dados_treino.category_1.unique():
    pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
    modelos[category] = pipe
    X_treino = dados_treino1.query(f'category_1 == "{category}"')['item_description']
    y_treino = np.log(dados_treino1.query(f'category_1 == "{category}"')['price'])
    # pipe.fit(X_treino, y_treino)
    modelos[category].fit(X_treino, y_treino)


Tempo de treino: 7m 18.3s

In [23]:
dados_teste1 = dados_teste1.assign(pred=0)

In [24]:
for category in dados_teste1.category_1.unique():
    index = dados_teste1.query(f'category_1 == "{category}"').index
    dados_teste1.loc[index, 'pred'] = np.exp(modelos[category].predict(dados_teste1.loc[index]['item_description']))

dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo de previsão: 18.8s

Avaliação geral

In [25]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.236
MAPE = 53.190 %
MAE = U$S 13.12
RMSE = U$S 33.54
MSLE = 0.352


Avaliação por categoria geral

In [26]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.218,0.290,0.157,0.276,0.097,0.158,0.116,0.161,0.269,0.185,0.033
MAPE,53.294,54.388,62.307,60.902,48.401,50.562,54.224,50.902,52.101,52.143,65.209
MAE,13.950,16.850,15.620,20.220,8.810,9.590,10.610,11.180,12.180,8.980,14.310
RMSE,34.600,33.770,46.560,56.000,23.030,20.410,25.820,21.340,25.390,22.610,29.780
MSLE,0.351,0.364,0.475,0.463,0.293,0.318,0.367,0.324,0.342,0.337,0.479


## Modelo geral usando comb_name_description

In [ ]:
dados_treino1 = dados_treino[['category_1', 'comb_name_description', 'price']]
dados_treino1.dropna(inplace=True)

dados_teste1 = dados_teste[['category_1', 'comb_name_description', 'price']]
dados_teste1.dropna(inplace=True)

In [28]:
pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
X_treino = dados_treino1['comb_name_description']
y_treino = np.log(dados_treino1['price'])
pipe.fit(X_treino, y_treino)

Pipeline(steps=[('tiidf',
                 TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 2))),
                ('xgboost',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                              predictor='auto', random_state=101, reg_alpha=0,
                              reg_lambda=1, ...))])

Tempo de treino: 10m 49.4s

In [ ]:
dados_teste1['pred'] = np.exp(pipe.predict(dados_teste1['comb_name_description']))
dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo previsão: 21.6s

Avaliação geral

In [30]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.229
MAPE = 51.808 %
MAE = U$S 12.86
RMSE = U$S 33.69
MSLE = 0.337


Avaliação por categoria

In [31]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.264,0.212,0.196,0.188,0.066,0.190,0.059,0.092,0.100,0.231,0.154
MAPE,46.377,44.691,68.584,61.543,55.121,55.156,63.779,51.797,51.862,67.867,55.190
MAE,12.950,17.010,15.450,21.010,9.270,9.630,11.340,11.540,13.040,9.360,12.740
RMSE,33.570,35.560,45.470,59.310,23.420,20.020,26.630,22.200,28.190,21.980,27.860
MSLE,0.298,0.371,0.482,0.496,0.316,0.312,0.416,0.341,0.379,0.363,0.386


## Modelo por categoria usando comb_name_description

In [32]:
modelos = {}

for category in dados_treino.category_1.unique():
    pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
    modelos[category] = pipe
    X_treino = dados_treino1.query(f'category_1 == "{category}"')['comb_name_description']
    y_treino = np.log(dados_treino1.query(f'category_1 == "{category}"')['price'])
    # pipe.fit(X_treino, y_treino)
    modelos[category].fit(X_treino, y_treino)


Tempo de treino: 8m 46.4s

In [33]:
dados_teste1 = dados_teste1.assign(pred=0)

In [34]:
for category in dados_teste1.category_1.unique():
    index = dados_teste1.query(f'category_1 == "{category}"').index
    dados_teste1.loc[index, 'pred'] = np.exp(modelos[category].predict(dados_teste1.loc[index]['comb_name_description']))

dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo previsão: 25.4s

Avaliação geral

In [35]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.320
MAPE = 46.681 %
MAE = U$S 11.85
RMSE = U$S 31.64
MSLE = 0.283


Avaliação por categoria principal

In [36]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.324,0.444,0.267,0.302,0.144,0.277,0.165,0.299,0.339,0.238,0.072
MAPE,45.714,46.247,56.942,54.254,43.903,45.331,49.776,44.999,46.405,46.513,59.973
MAE,12.380,14.710,14.470,18.830,8.240,8.770,9.990,10.080,11.090,8.340,13.560
RMSE,32.170,29.890,43.410,54.970,22.430,18.910,25.100,19.510,24.160,21.860,29.170
MSLE,0.271,0.272,0.406,0.380,0.250,0.263,0.324,0.264,0.288,0.279,0.437
